## GitHub Data Analysis


## Introduction
Every software engineer uses GitHub. Being a open source distributed version control tool, GitHub has thoundsands of new repositorys in every hour. Thus, GitHub could also be used as a huge dynamic data source to analyze technology status quo and trend.

In this project, we will be looking into serveral things like who is the most popular person in certain field, what is the current hottest project and how much does different programming languages being used.


### GitHub API

We will use GitHub API from [here](https://developer.github.com/v3/).
All the API calls are using HTTPS requests and it will return in JSON format.

Steps to use GitHub API:
1. Install `pygithub` by 
`-pip install pygithub`
2. Generate a GitHub Personal access token required for `GitHub API`
3. Test You API in local terminal using the following command. It is expected to return a list of dictionary contains your account info
##### - curl https://api.github.com/user\?access_token\={YOUR_TOKEN}




### NetworkX
NetworkX is a Python package for the creation, manipulation, and study of the structure, dynamics, and functions of complex networks. It provides tools to work with large dataset with network strucutres. With NetworkX, we can load and store neyworks in standard data format easily. It can also helps us to generate classic networks, analyze network strucutre, build network models and much more.

You can install `NetworkX` by `-pip install networkx`


In [6]:
import sys
from github import Github
import networkx as nx
from operator import itemgetter

# Global Variables
ACCESS_TOKEN = '05bb4eb867b152be20dd11f4fa292107c839931c'
USER = 'minrk'  # Define the GitHub User Name
REPO = 'findspark'  # Define the Repo name
client = Github(ACCESS_TOKEN)
graph = nx.DiGraph()

### Set Up NetworkX Graph


After defined the user and repo name that we are going to explore, we can then set up the NetworkX graph.

We will add the repo and each user who starred the repo as nodes, and build edges between them. After this, we also add edges between users and their followers. 

In [7]:
def buildRepoRelations(REPO):
    user = client.get_user(USER)
    repo = user.get_repo(REPO)  # Get a specific repo
    REPOS = user.get_repos()
    stargazers = list(repo.get_stargazers())  # The list of users who starred this REPO
    graph.add_node(repo.name + '(repo)', type='repo', lang=repo.language, owner=user.login)
    for stargazer in stargazers:
        graph.add_node(stargazer.login + '(user)', type='user')
        graph.add_edge(stargazer.login + '(user)', repo.name + '(repo)', type='gazes')
    #	print(len(stargazers))#See if it return a correct list
    return stargazers


def buildUserRelations(stargazers):
    for i, stargazer in enumerate(stargazers):
        followers = stargazer.get_followers()
        try:
            for follower in followers:
                if follower.login + '(user)' in graph:
                    graph.add_edge(follower.login + '(user)', stargazer.login + '(user)', type='follows')
        except Exception:  
            print("Encountered an error when finding follower for user: ", stargazer.login)
        #See How many available API calls remaining
        print ("API Calls Remaining", client.rate_limiting)

In [8]:
stargazers = buildRepoRelations(REPO)
buildUserRelations(stargazers)

API Calls Remaining (4981, 5000)
API Calls Remaining (4956, 5000)
API Calls Remaining (4953, 5000)
API Calls Remaining (4951, 5000)
API Calls Remaining (4948, 5000)
API Calls Remaining (4930, 5000)
API Calls Remaining (4928, 5000)
API Calls Remaining (4927, 5000)
API Calls Remaining (4918, 5000)
API Calls Remaining (4915, 5000)
API Calls Remaining (4914, 5000)
API Calls Remaining (4909, 5000)
API Calls Remaining (4907, 5000)
API Calls Remaining (4906, 5000)
API Calls Remaining (4905, 5000)
API Calls Remaining (4904, 5000)
API Calls Remaining (4895, 5000)
API Calls Remaining (4894, 5000)
API Calls Remaining (4893, 5000)
API Calls Remaining (4891, 5000)
API Calls Remaining (4889, 5000)
API Calls Remaining (4887, 5000)
API Calls Remaining (4886, 5000)
API Calls Remaining (4885, 5000)
API Calls Remaining (4884, 5000)
API Calls Remaining (4883, 5000)
API Calls Remaining (4882, 5000)
API Calls Remaining (4879, 5000)
API Calls Remaining (4878, 5000)
API Calls Remaining (4877, 5000)
API Calls 

### Find Hottest User
In this step, we use the graph initialized above to find the hottest users. The hottest user is defined as the GitHub user followed by most of the people who starred the repo we defined previously. This can also be interpreted as those who starred this repo also follows ... 


In [9]:
from collections import Counter
from operator import itemgetter

def getHottestUser(stargazers):
    
    temp_list = []
    for edge in graph.edges(data = True):
        if edge[2]['type'] == 'follows':
            temp_list.append(edge[1])
    counter = Counter(temp_list)
    
    popular_users = []
    for u, f in counter.most_common():
        popular_users.append((u,f))
    print ("Number of popular users", len(popular_users))
    print ("Top popular users:", popular_users[:10])
    
getHottestUser(stargazers)




Number of popular users 32
Top popular users: [('minimaxir(user)', 7), ('stared(user)', 6), ('freeman-lab(user)', 5), ('rgbkrk(user)', 4), ('nchammas(user)', 3), ('dclambert(user)', 3), ('esafak(user)', 3), ('dapurv5(user)', 2), ('rholder(user)', 2), ('xiaohan2012(user)', 2)]


The result above shows the most popular users. However, we care more about some centralities that NetworkX provided.
#### Degree Centrality
First, the Degree Centrality for a node v is the fraction of nodes it is connected to. 
#### Betweenness Centrality
Also, the Betweenness Centrality compute the shortest path for nodes. It is the sum of the fraction of all-pairs shortest paths that pass through the node v. 
#### Closeness Centrality
Lastly, the Closeness Centrality of a node u is the reciprocal of the sum of the shortest path distances from u to all n-1 other nodes. Since the sum of distances depends on the number of nodes in the graph, closeness is normalized by the sum of minimum possible distances n-1.




In [46]:
def formatResult(graph):
    graph_copy = graph.copy()
    # Remove center node
    graph_copy.remove_node('findspark(repo)')

    dc = sorted(nx.degree_centrality(graph_copy).items(), 
                key=itemgetter(1), reverse=True)

    bc = sorted(nx.betweenness_centrality(graph_copy).items(), 
                key=itemgetter(1), reverse=True)
    cc = sorted(nx.closeness_centrality(graph_copy).items(), 
                key=itemgetter(1), reverse=True)
    return (dc, bc, cc)

dc, bc, cc = formatResult(graph)

print ("Degree Centrality")
print (dc[:5],'\n')

print ("Betweenness Centrality")
print (bc[:5],'\n')

print ("Closeness Centrality")
print (cc[:5])

Number of popular users 32
Top popular users: [('minimaxir(user)', 7), ('stared(user)', 6), ('freeman-lab(user)', 5), ('rgbkrk(user)', 4), ('nchammas(user)', 3), ('dclambert(user)', 3), ('esafak(user)', 3), ('rholder(user)', 2), ('jackmaney(user)', 2), ('amontalenti(user)', 2)]
Degree Centrality
[('fly51fly(user)', 0.09090909090909091), ('gauravssnl(user)', 0.045454545454545456), ('esafak(user)', 0.04040404040404041), ('andrewiiird(user)', 0.03535353535353536), ('minimaxir(user)', 0.03535353535353536)] 

Betweenness Centrality
[('esafak(user)', 0.0003888290690321147), ('rgbkrk(user)', 0.00029482643695841665), ('dapurv5(user)', 0.00010254832589857971), ('andrewiiird(user)', 9.400263207369807e-05), ('nchammas(user)', 8.972978516125725e-05)] 

Closeness Centrality
[('freeman-lab(user)', 0.03711023276240668), ('minimaxir(user)', 0.03535353535353535), ('stared(user)', 0.030934343434343432), ('rgbkrk(user)', 0.02693602693602694), ('nchammas(user)', 0.0202020202020202)]


### Find Hottest Repository 

Next, we go through each user for their starred repos and then add these repos into the network. After that, it is easy for us to get the popular repositories. Moreover, we can also get to know the language preference of one certain user.

In [43]:
def buildRepoNet(stargazers, limit_repo):
    for i, v in enumerate(stargazers):
        print(v.login)
        try:
            for starred in v.get_starred()[:limit_repo]:  # Slice to avoid supernodes
                graph.add_node(starred.name + '(repo)', type='repo', lang=starred.language, \
                           owner=starred.owner.login)
                graph.add_edge(v.login + '(user)', starred.name + '(repo)', type='gazes')
        except Exception:  # ssl.SSLError:
            print("Encountered an error fetching starred repos for", v.login, "Skipping.")

        print("Num nodes/edges in graph", graph.number_of_nodes(), "/", graph.number_of_edges())
    print(nx.info(graph), '\n')


Sometimes a user marks too many repos and it takes a lot of time to build the net. So here the limit_repo parameter could define the maximum of the repos of one user

In [44]:
buildRepoNet(stargazers,5)

pchalasani
Num nodes/edges in graph 2740 / 2920
rgbkrk
Num nodes/edges in graph 2740 / 2920
esafak
Num nodes/edges in graph 2740 / 2920
rdhyee
Num nodes/edges in graph 2740 / 2920
rholder
Num nodes/edges in graph 2740 / 2920
freeman-lab
Num nodes/edges in graph 2740 / 2920
he0x
Num nodes/edges in graph 2740 / 2920
cbouey
Num nodes/edges in graph 2740 / 2920
stared
Num nodes/edges in graph 2740 / 2920
ryan-williams
Num nodes/edges in graph 2740 / 2920
nehalecky
Num nodes/edges in graph 2740 / 2920
jazzwang
Num nodes/edges in graph 2740 / 2920
paulochf
Num nodes/edges in graph 2740 / 2920
Erstwild
Num nodes/edges in graph 2740 / 2920
sandysnunes
Num nodes/edges in graph 2740 / 2920
d18s
Num nodes/edges in graph 2740 / 2920
amontalenti
Num nodes/edges in graph 2740 / 2920
richardskim111
Num nodes/edges in graph 2740 / 2920
hmourit
Num nodes/edges in graph 2740 / 2920
robcowie
Num nodes/edges in graph 2740 / 2920
ocanbascil
Num nodes/edges in graph 2740 / 2920
fish2000
Num nodes/edges in g

Num nodes/edges in graph 2745 / 2927
0asa
Num nodes/edges in graph 2745 / 2927
whtsky
Num nodes/edges in graph 2745 / 2927
freegnu
Num nodes/edges in graph 2745 / 2927
SoyGema
Num nodes/edges in graph 2745 / 2927
adonovan7
Num nodes/edges in graph 2745 / 2927
prometheosduan
Num nodes/edges in graph 2745 / 2927
sourabhd
Num nodes/edges in graph 2745 / 2927
mikestaszel
Num nodes/edges in graph 2745 / 2927
sophiedophie
Num nodes/edges in graph 2745 / 2927
Nimayer
Num nodes/edges in graph 2745 / 2927
chocolocked
Num nodes/edges in graph 2745 / 2927
gabides
Num nodes/edges in graph 2745 / 2927
soumyadsanyal
Num nodes/edges in graph 2745 / 2927
mas-dse-m2ryu
Num nodes/edges in graph 2745 / 2927
Boukefalos
Num nodes/edges in graph 2745 / 2927
saikatkumardey
Num nodes/edges in graph 2745 / 2927
pluteski
Num nodes/edges in graph 2745 / 2927
rainsunny
Num nodes/edges in graph 2745 / 2927
shriharishastry
Num nodes/edges in graph 2745 / 2927
bwhitesell
Num nodes/edges in graph 2745 / 2927
fivejjs


In [45]:
def getTopNRepos(n):
    print("Top "+str(n)+" Popular repositories:")
    
    repos = []
    for (v, i) in graph.in_degree_iter():
        if graph.node[v]['type'] == 'repo':
            repos.append((v,i))
    repos = sorted(repos, key = lambda x:x[1], reverse=True)
    print(repos[:n])

In [46]:
getTopNRepos(10)

Top 10 Popular repositories:
[('findspark(repo)', 199), ('foundationdb(repo)', 5), ('kubeflow(repo)', 4), ('cpython(repo)', 4), ('Fast-Pandas(repo)', 4), ('arrow(repo)', 3), ('luigi(repo)', 3), ('keras(repo)', 3), ('phoenix(repo)', 3), ('xgboost(repo)', 3)]


In [47]:
def getUserPreference(username):
    print("Respositories that "+ username+" has starred")
    for v in graph[username+"(user)"]:
        if graph[username+"(user)"][v]['type'] == 'gazes':
            print(v)

    print("Programming languages "+ username+" is interested in")
    
    langs = set()
    for v in graph[username+"(user)"]:
        if graph[username+"(user)"][v]['type'] == 'gazes':
            langs.add(graph.node[v]['lang'])
    print(langs)




In [48]:
getUserPreference('luzhijun')

Respositories that luzhijun has starred
findspark(repo)
Kalman-and-Bayesian-Filters-in-Python(repo)
TensorFlow-Machine-Learning-Cookbook(repo)
recharts2(repo)
fpinscala(repo)
be-a-professional-programmer(repo)
Programming languages luzhijun is interested in
{None, 'Scala', 'Python', 'R', 'Jupyter Notebook'}
